In [12]:
# written by Yosia E. Lukumai
# loading the modules
import torch
import numpy as np
import cv2
import time

In [ ]:
class helmetDetetector:
    """
    Custom class for deploying the yolo modal
    """
    
    def __init__(self,mode: int, model_name=str, imagePath=None):
        """
        Initializes the class with youtube url and output file.
        :param mode: define whether is image format or video
        :param url: Has to be as youtube URL,on which prediction is made.
        :param out_file: A valid output file name.
        """
        self.model = self.load_model(model_name)
        self.mode = mode
        self.classes = self.model.names
        self.device = 'cuda' if torch.cuda.is_available() else 'cpu'
        self.imagePath = imagePath
        print("\n\nDevice Uses:",self.device)



    def load_model(self, model_name):
        """
        Loads Yolo5 model from pytorch hub.
        :return: Trained Pytorch model.
        """
        # pass the pass where the ultraytic/yolov5 repository is found
        if model_name:
#             model = torch.hub.load('ultralytics/yolov5', 'custom', path=model_name, force_reload=True)
            model = torch.hub.load('yolov5', 'custom', path=model_name, source="local", force_reload=True)
        else:
             model = torch.hub.load('ultralytics/yolov5', 'yolov5s', pretrained=True)
        return model


    def score_frame(self, frame):
        """
        Takes a single frame as input, and scores the frame using yolo5 model.
        :param frame: input frame in numpy/list/tuple format.
        :return: Labels and Coordinates of objects detected by model in the frame.
        """
        self.model.to(self.device)
        frame = [frame]
        results = self.model(frame)
     
        labels, cord = results.xyxyn[0][:, -1], results.xyxyn[0][:, :-1]
        return labels, cord


    def class_to_label(self, x):
        """
        For a given label value, return corresponding string label.
        :param x: numeric label
        :return: corresponding string label
        """
        return self.classes[int(x)]


    def plot_boxes(self, results, frame):
        """
        Takes a frame and its results as input, and plots the bounding boxes and label on to the frame.
        :param results: contains labels and coordinates predicted by model on the given frame.
        :param frame: Frame which has been scored.
        :return: Frame with bounding boxes and labels ploted on it.
        """
        labels, cord = results
        n = len(labels)
        x_shape, y_shape = frame.shape[1], frame.shape[0]
        for i in range(n):
            row = cord[i]
            if row[4] >= 0.2:
                x1, y1, x2, y2 = int(row[0]*x_shape), int(row[1]*y_shape), int(row[2]*x_shape), int(row[3]*y_shape)
                bgr = (0, 255, 0)
                cv2.rectangle(frame, (x1, y1), (x2, y2), bgr, 2)
                cv2.putText(frame, "Helmet", (x1, y1), cv2.FONT_HERSHEY_SIMPLEX, 0.9, bgr, 2)

        return frame
    


    def __call__(self):
        """
        This function is called when class is executed, it runs the loop to read the video frame by frame,
        and write the output into a new file.
        :return: void
        """
        if self.mode == 1:
            cap = cv2.VideoCapture("SampleVideos/sample4.mp4")

            while cap.isOpened():

                start_time = time.perf_counter()
                ret, frame = cap.read()
                if not ret:
                    break
                results = self.score_frame(frame)
                frame = self.plot_boxes(results, frame)
                end_time = time.perf_counter()
                fps = 1 / np.round(end_time - start_time, 3)
                cv2.putText(frame, f'FPS: {int(fps)}', (20,70), cv2.FONT_HERSHEY_SIMPLEX, 1.5, (0,255,0), 2)
                cv2.imshow("img", frame)
                

                if cv2.waitKey(1) & 0xFF == ord('q'):
                    break
            cv2.destroyAllWindows()
            cap.release()
                    
        elif self.mode == 2:
            # picture mode
            img = cv2.imread(self.imagePath)
            results = self.score_frame(img)
            img = self.plot_boxes(results, img)
            cv2.imshow("img", img)      
            cv2.waitKey(0)
            cv2.destroyAllWindows()
        else:
            return
            
            


In [ ]:
# Create a new object and execute.
# passing the image
detection = helmetDetetector(2, "best.pt","SamplePics/3.jpg")
detection()

# passing the image pass the sample of videos in the call dunder function
# detection = helmetDetetector(2, "best.pt","SamplePics/3.jpg")
# detection()